# Testing Layers

In [ ]:
import os
import time

from felt_python import (
    create_map,
    delete_map,
    get_map_details,
    list_layers,
    upload_file,
    upload_url,
    upload_dataframe,
    upload_geodataframe,
    refresh_file_layer,
    refresh_url_layer,
    get_layer_details,
    update_layer_style
)

os.environ["FELT_API_TOKEN"] = "<YOUR_API_TOKEN>"

# Maps

Create a map and retrieve its details, storing the ID in a variable

In [ ]:
resp = create_map(
    title="A felt-py map",
    lat=40,
    lon=-3,
    public_access="private",
)
map_id = resp["id"]

In [ ]:
get_map_details(map_id)

# Layers

Create layers from file, URL and (Geo)DataFrame uploads

## Upload a file

In [ ]:
layer_resp = upload_file(map_id, "tests/fixtures/null-island-points-sample.geojson", "The Points Layer")
layer_id = layer_resp["layer_id"]

# Check upload status

In [ ]:
while get_layer_details(map_id, layer_id)["progress"] < 100:
    print("Waiting for layer to finish processing...")
    time.sleep(5)

### Refresh file upload

Wait for upload to finish first

In [ ]:
refresh_file_layer(map_id, layer_id, file_name="tests/fixtures/null-island-points.geojson")

## Upload a URL

In [ ]:
live_earthquakes_url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_hour.geojson"
url_upload = upload_url(map_id, live_earthquakes_url, "Live Earthquakes")
url_layer_id = url_upload["layer_id"]

### Refresh URL upload

Wait for upload to finish first

In [ ]:
while get_layer_details(map_id, url_layer_id)["progress"] < 100:
    print("Waiting for layer to finish processing...")
    time.sleep(5)
refresh_url_layer(map_id, url_layer_id)

# Styling

Fetch style of file upload layer

In [ ]:
current_style = get_layer_details(map_id, layer_id)["style"]
current_style

Update style

In [ ]:
new_style = current_style.copy()
new_style["style"]["color"] = "red"
new_style["style"]["size"] = 20
update_layer_style(map_id, layer_id, new_style)

# DataFrames and GeoDataFrames

In [ ]:
import pandas as pd

df = pd.read_csv("tests/fixtures/null-island-polygons-wkt.csv")
upload_dataframe(map_id, df, "Polygons from a CSV")

In [ ]:
import geopandas as gpd

gdf = gpd.read_file('tests/fixtures/null-island-points.geojson')
upload_geodataframe(map_id, gdf, layer_name="GeoPandas Upload")

## List all uploaded layers

In [ ]:
list_layers(map_id)

# Downloading a layer

Download the file uploaded layer as a GeoPackage

In [ ]:
from felt_python import download_layer

download_layer(map_id, layer_id, file_name="exported.gpkg")

# Cleanup

In [ ]:
delete_map(map_id)